In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [24]:
class MyActivation(keras.layers.Layer):
    def __init__(self, supports_masking=True, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        # Signal that the layer is safe for mask propagation
        self.supports_masking = supports_masking

    def call(self, inputs, mask=None):
        tf.print('call mask', mask)
        return tf.nn.relu(inputs)

    def compute_mask(self, inputs, mask=None):
        return tf.equal(inputs, 0)

class MyActivation2(keras.layers.Layer):
    def __init__(self, supports_masking=True, **kwargs):
        super(MyActivation2, self).__init__(**kwargs)
        # Signal that the layer is safe for mask propagation
        self.supports_masking = supports_masking
        self.rr = MyActivation(supports_masking=True)

    def call(self, inputs, mask=None):
        tf.print('call mask', mask)
        inputs = tf.nn.relu(inputs)
        # inputs = self.rr(inputs)
        # tf.print(inputs._keras_mask)
        return inputs

    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, 1)

In [25]:
inputs = np.random.randint(0, 10, size=(2, 5))
x = layers.Embedding(input_dim=5000, output_dim=3, mask_zero=True)(inputs)
x._keras_mask

<tf.Tensor: shape=(2, 5), dtype=bool, numpy=
array([[False,  True,  True,  True,  True],
       [ True,  True,  True,  True, False]])>

In [26]:
inputs

array([[0, 6, 6, 4, 9],
       [1, 9, 6, 3, 0]])

In [27]:
x

<tf.Tensor: shape=(2, 5, 3), dtype=float32, numpy=
array([[[-0.0097435 ,  0.04821042, -0.04418812],
        [-0.02169017, -0.03551453, -0.02851381],
        [-0.02169017, -0.03551453, -0.02851381],
        [-0.02520925,  0.02708708, -0.03410064],
        [ 0.04544548, -0.04514576, -0.04039621]],

       [[-0.00302394, -0.04879255, -0.00579654],
        [ 0.04544548, -0.04514576, -0.04039621],
        [-0.02169017, -0.03551453, -0.02851381],
        [ 0.00075462,  0.04632522, -0.00729251],
        [-0.0097435 ,  0.04821042, -0.04418812]]], dtype=float32)>

when a layer is not supporting masking, even if
the input tensor (x) has a mask, it won't be transferred.
You can see the mask in the layer's call method is None.
Even if the layer has a compute_mask method it won't take effect.

In [28]:
x = MyActivation2(supports_masking=False)(x)  # Will pass the mask along
print("Mask not found in output:", x._keras_mask)

call mask [[0 1 1 1 1]
 [1 1 1 1 0]]


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute '_keras_mask'

In [29]:
x

<tf.Tensor: shape=(2, 5, 3), dtype=float32, numpy=
array([[[0.        , 0.04821042, 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.02708708, 0.        ],
        [0.04544548, 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.04544548, 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.00075462, 0.04632522, 0.        ],
        [0.        , 0.04821042, 0.        ]]], dtype=float32)>

you can see input x has no mask, so the mask in layer's call method is None.
But because the layer is supporting masking, so the compute_mask method will
take effect to produce a mask for the output tensor.


In [30]:
x = MyActivation(supports_masking=True)(x)  #
print("Mask found:", x._keras_mask)

call mask None
Mask found: tf.Tensor(
[[[ True False  True]
  [ True  True  True]
  [ True  True  True]
  [ True False  True]
  [False  True  True]]

 [[ True  True  True]
  [False  True  True]
  [ True  True  True]
  [False False  True]
  [ True False  True]]], shape=(2, 5, 3), dtype=bool)


In [31]:
x

<tf.Tensor: shape=(2, 5, 3), dtype=float32, numpy=
array([[[0.        , 0.04821042, 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.02708708, 0.        ],
        [0.04544548, 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.04544548, 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.00075462, 0.04632522, 0.        ],
        [0.        , 0.04821042, 0.        ]]], dtype=float32)>

In [36]:
adapt_data = np.array([[0., 7., 4.],
                       [2., 9., 6.],
                       [0., 7., 4.],
                       [2., 9., 6.]], dtype='float32')
input_data = np.array([[0., 7., 4.]], dtype='float32')
layer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
layer.adapt(adapt_data)

In [45]:
np.mean(adapt_data, axis=0)

array([1., 8., 5.], dtype=float32)

In [42]:
layer.mean

<tf.Variable 'mean:0' shape=(3,) dtype=float32, numpy=array([1., 8., 5.], dtype=float32)>

In [40]:
layer.variance

<tf.Variable 'variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

In [12]:
layer(input_data)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-1., -1., -1.]], dtype=float32)>